# Zadanie bonusowe do PD4
### Mateusz Krzyziński, grupa 2


W tym zadaniu po raz kolejny wykrozystamy zbiór danych z PD2 (doyczący sprzedawanych na Allegro produktów). Będziemy uczyć modele regresji do prognozowania zmiennej price na podstawie zmiennych objaśniających: main_category, categories i it_location.

W zakres zadania wchodzą:
- zastosowanie target encodingu (należy przemyśleć parametr smoothing i wyjaśnić dlaczego jest on ważny)
- nauczenie modelu liniowego do przewidywania ceny produktu
- zastosowanie co najmniej dwóch wariantów regularyzacji i przeanalizowanie ich wpływu na jakość predykcji 
- nauczenie innego modelu regresyjnego
- wykorzystanie miary RMSE i R2 do wybrania najlepszego wariantu modelu


#### Import pakietów

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#### Wczytanie danych

In [4]:
df = pd.read_csv("https://www.dropbox.com/s/360xhh2d9lnaek3/allegro-api-transactions.csv?dl=1")

In [5]:
# ograniczamy ramkę do interesujących nas zmiennych
df = df[['main_category', 'categories', 'it_location', 'price']]

In [7]:
df

,main_category,categories,it_location,price
0,Komputery,"['Komputery', 'Dyski i napędy', 'Nośniki', 'No...",Warszawa,59.99
1,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Bielizna damska',...",Warszawa,4.90
2,Dom i Ogród,"['Dom i Ogród', 'Budownictwo i Akcesoria', 'Śc...",Leszno,109.90
3,Książki i Komiksy,"['Książki i Komiksy', 'Poradniki i albumy', 'Z...",Wola Krzysztoporska,18.50
4,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Ślub i wesele', '...",BIAŁYSTOK,19.90
...,...,...,...,...
420015,RTV i AGD,"['RTV i AGD', 'Sprzęt audio dla domu', 'Odtwar...",Kraśnik,180.00
420016,Uroda,"['Uroda', 'Makijaż', 'Oczy', 'Tusze do rzęs']",Dzierżoniów,14.99
420017,"Odzież, Obuwie, Dodatki","['Odzież, Obuwie, Dodatki', 'Przebrania, kosti...",Supraśl,5.99
420018,Dla Dzieci,"['Dla Dzieci', 'Rowery i pojazdy', 'Rowery bie...",Poznań,200.00


In [15]:
df.info() #nie ma braków danych

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420020 entries, 0 to 420019
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   main_category  420020 non-null  object 
 1   categories     420020 non-null  object 
 2   it_location    420020 non-null  object 
 3   price          420020 non-null  float64
dtypes: float64(1), object(3)
memory usage: 12.8+ MB


Zmienne, na podstawie których będziemy przewidywać cenę są zmiennymi kategorycznymi, w związku z tym będziemy je musieli zakodować w sposób numeryczny. Wykorzystamy do tego target encoding, wymagany w poleceniu zadania. 

### Target encoding 

Zanim wykonamy encoding przyjrzyjmy się najpierw kolumnom, które będziemy kodować. 

In [10]:
df[['main_category', 'categories', 'it_location']].describe()

,main_category,categories,it_location
count,420020,420020,420020
unique,27,9020,10056
top,Dom i Ogród,"['Dom i Ogród', 'Ogród', 'Rośliny', 'Rośliny o...",Warszawa
freq,91042,3753,23244


W kolumnach categories i it_location mamy odpowiednio około 9 i 10 tysięcy unikalnych wartości. Zatem rzeczywiście target encoding jest odpowiednim rozwiązaniem (stosując np. one hot encoding stworzylibyśmy za dużo nowych kolumn). 

Na podstawie wiedzy z zadania PD2 wiemy, że możemy ograniczyć liczbę unikalnych wartości w kolumnie it_location. Nazwy miejscowości są zapisane bowiem w różny sposób (od wielkiej litery, tylko wielkimi literami etc.) - lokalizacja była zapewne wpisywana przez sprzedawców, a nie wybierana z listy. W związku z tym w celu ujednolicenia sposobu zapisu możemy zamienić wszystko na małe litery.

In [6]:
df["it_location"] = df["it_location"].str.lower()

In [7]:
df["it_location"].describe()

count       420020
unique        7903
top       warszawa
freq         27042
Name: it_location, dtype: object

Teraz mamy już ponad 2000 mniej unikalnych wartości i nie tworzymy sztucznych podziałów.

Chcąc przeprowadzić target encoding, musimy mieć na uwadze, że taki sposób kodowania zmiennych kategorycznych ma swoje wady: 
- zależność od zmiennej celu może prowadzić do przeuczania się modelu,
- tracimy informacje o poszczególnych kategoriach (są one niejako grupowane względem odpowiedniej średniej), 
- w kategoriach o małej liczności średnia jest słabym parametrem i łatwo doprowadzić do overfittingu. 

W tym przypadku mamy wiele unikalnych wartości zmiennej kategorycznej i część z nich ma małą liczność. W takim wypadku skorzystanie ze zwykłej średniej z targetu może nie odzwierciedlać dobrze sytuacji. Na zbiorze testowym wynik będzie dobry, ale na testowym model nie poradzi sobie zbyt dobrze.

Dlatego należy zastosować smoothing - wygładzanie. Gdy próbka jest zbyt mała, nie bierzemy jej średniej wprost, tylko wartość estymujemy z wykorzystaniem średniej globalnej, z całego zbioru. W ten sposób etykieta przyporządkowana obserwacjom z danej grupy jest obliczana na podstawie wzoru:
$$ S_i = \lambda(n_i) \cdot \mu_i + (1 - \lambda(n_i)) \cdot \mu_X,$$ 
gdzie:
- $n_i$ oznacza ilość obserwacji w $i$-tej grupie, 
- $\mu_i$ - średnią z targetu dla obserwacji w $i$-tej grupie,  
- $\mu_X$ - średnią z targetu dla wszystkich obserwacji ze zbioru, 
- $\lambda(n_i)$ jest rosnącą funkcją wielkości próbki $n_i$ o wartościach z zakresu $[0, 1]$.

W implementacji TargetEncoder w pakiecie category_encoders funkcja lamda jest zdefiniowana jako: 
$$ \lambda(n) = \frac{1}{1 + exp(\frac{-(n-k)}{f}) }  ,$$
gdzie:
- $n$ to wielkość próbki w danej grupie, 
- $k$ to parametr min_samples_leaf - minimalną liczbę obserwacji w grupie, żeby wziąć ją pod uwagę,
- $f$ to parametr smoothing. 

Zatem i większy parametr smoothing, tym większa regularyzacja - mniejsze znaczenie ma średnia z targetu dla samej kategorii. 




Źródło: https://www.researchgate.net/publication/220520258_A_Preprocessing_Scheme_for_High-Cardinality_Categorical_Attributes_in_Classification_and_Prediction_Problems

Spójrzmy jeszcze na to, jak wyglądają kolumny, które chcemy kodować w ten sposób. 

In [9]:
df['main_category'].value_counts()

Dom i Ogród                            91042
Odzież, Obuwie, Dodatki                54257
Motoryzacja                            45941
Dla Dzieci                             42107
Uroda                                  28096
Sport i Turystyka                      27532
RTV i AGD                              20341
Telefony i Akcesoria                   19805
Komputery                              14491
Zdrowie                                13166
Książki i Komiksy                      11572
Delikatesy                              8074
Gry                                     7150
Rękodzieło                              6574
Kolekcje                                6146
Przemysł                                5959
Biżuteria i Zegarki                     5808
Biuro i Reklama                         3194
Fotografia                              2381
Muzyka                                  1961
Antyki i Sztuka                         1214
Konsole i automaty                      1053
Filmy     

In [12]:
df['categories'].value_counts() #widzimy, że nie trzeba ich rozdzielać, ciąg kategorii jednoznacznie ją definiuje 
#to zmienna z największą liczbą podgrup, w tym wiele z pojedynczą obserwacją

['Dom i Ogród', 'Ogród', 'Rośliny', 'Rośliny owocowe']                                 3753
['Dom i Ogród', 'Ogród', 'Rośliny', 'Drzewa i krzewy liściaste']                       2914
['Dom i Ogród', 'Ogród', 'Rośliny', 'Bulwy, cebulki, kłącza']                          2693
['Odzież, Obuwie, Dodatki', 'Odzież damska', 'Sukienki']                               2485
['Motoryzacja', 'Opony', 'Opony samochodowe', 'Letnie']                                2473
                                                                                       ... 
['Konsole i automaty', 'Nintendo (SNES i NES)', 'NES']                                    1
['Kolekcje', 'Numizmatyka', 'Ameryka Pn. i Pd.', 'USA', 'obiegowe', 'do 1945 roku']       1
['Kolekcje', 'Pieniądz papierowy', 'Afryka', 'Erytrea']                                   1
['RTV i AGD', 'Sprzęt car audio', 'Radioodtwarzacze', 'CD', 'Fabryczne', 'BMW']           1
['Komputery', 'Podzespoły bazowe', 'Płyty główne', 'Socket FM2']                

In [13]:
df['it_location'].value_counts()

warszawa                             27042
kraków                               16581
łódź                                 12433
poznań                               11197
internet                             10992
                                     ...  
głogów małopolski, rudna mała 160        1
jegłownik                                1
gdów/kraków                              1
gorzów okolice                           1
wrocław, kazanów gm. strzelin            1
Name: it_location, Length: 7903, dtype: int64

## Modelowanie

In [124]:
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score

### Regresja liniowa

In [112]:
X = df.drop("price", axis=1)
y = df["price"]

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [120]:
lr = Pipeline(steps=[('encoder', TargetEncoder()),  
                     ('LinReg', LinearRegression())])  

param_grid = {'encoder__smoothing': [0.01, 0.1, 1., 10., 100., 1000.],
              'encoder__min_samples_leaf': [1, 2, 3]}


lr_tuned = GridSearchCV(lr, param_grid,
                          cv = 3, 
                          scoring = ['neg_root_mean_squared_error', 'r2'], 
                          refit = 'neg_root_mean_squared_error',
                          verbose = 3)

lr_tuned.fit(X_train, y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.01 ............


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.01, neg_root_mean_squared_error=-516.221, r2=0.058, total=   1.7s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.01 ............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.01, neg_root_mean_squared_error=-348.576, r2=0.116, total=   1.6s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.01 ............


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s


[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.01, neg_root_mean_squared_error=-243.179, r2=0.187, total=   1.6s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.1 .............
[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.1, neg_root_mean_squared_error=-516.221, r2=0.058, total=   1.7s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.1 .............
[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.1, neg_root_mean_squared_error=-348.576, r2=0.116, total=   1.8s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=0.1 .............
[CV]  encoder__min_samples_leaf=1, encoder__smoothing=0.1, neg_root_mean_squared_error=-243.179, r2=0.187, total=   1.7s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=1.0 .............
[CV]  encoder__min_samples_leaf=1, encoder__smoothing=1.0, neg_root_mean_squared_error=-516.983, r2=0.055, total=   1.6s
[CV] encoder__min_samples_leaf=1, encoder__smoothing=1.0 .............
[CV]  encoder__min_samples_leaf=1, enco

[CV]  encoder__min_samples_leaf=3, encoder__smoothing=10.0, neg_root_mean_squared_error=-518.341, r2=0.050, total=   1.6s
[CV] encoder__min_samples_leaf=3, encoder__smoothing=10.0 ............
[CV]  encoder__min_samples_leaf=3, encoder__smoothing=10.0, neg_root_mean_squared_error=-349.572, r2=0.111, total=   1.8s
[CV] encoder__min_samples_leaf=3, encoder__smoothing=10.0 ............
[CV]  encoder__min_samples_leaf=3, encoder__smoothing=10.0, neg_root_mean_squared_error=-243.824, r2=0.183, total=   1.7s
[CV] encoder__min_samples_leaf=3, encoder__smoothing=100.0 ...........
[CV]  encoder__min_samples_leaf=3, encoder__smoothing=100.0, neg_root_mean_squared_error=-517.872, r2=0.052, total=   2.4s
[CV] encoder__min_samples_leaf=3, encoder__smoothing=100.0 ...........
[CV]  encoder__min_samples_leaf=3, encoder__smoothing=100.0, neg_root_mean_squared_error=-350.234, r2=0.107, total=   2.0s
[CV] encoder__min_samples_leaf=3, encoder__smoothing=100.0 ...........
[CV]  encoder__min_samples_leaf=3

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  1.6min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('encoder',
                                        TargetEncoder(cols=None,
                                                      drop_invariant=False,
                                                      handle_missing='value',
                                                      handle_unknown='value',
                                                      min_samples_leaf=1,
                                                      return_df=True,
                                                      smoothing=1.0,
                                                      verbose=0)),
                                       ('LinReg',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                        

In [133]:
lr_tuned.best_score_

-368.4911705309187

In [150]:
lr_tuned.best_params_

{'encoder__min_samples_leaf': 3, 'encoder__smoothing': 0.1}

In [122]:
y_hat_lr_tuned = lr_tuned.predict(X_test)
print(f"""RMSE: {mean_squared_error(y_test, y_hat_lr_tuned, squared=False):.4f}
R^2 score: {r2_score(y_test, y_hat_lr_tuned):.4f}""")

RMSE: 171.0608
R^2 score: 0.3383


### Ridge 
Regresja z regularyzacją L2 (do funkcji straty dodawany jest składnik ograniczający wartość wag $\lambda||w||^2$, gdzie $||w||^2$ to suma kwadratów współczynników i $\lambda$ jest parametrem). 

In [128]:
ridge = Pipeline(steps=[('encoder', TargetEncoder()),  
                     ('Ridge', Ridge())])  


param_grid = {'encoder__smoothing': [0.1, 1., 10., 100., 1000.],
              'encoder__min_samples_leaf': [1, 2, 3],
              'Ridge__alpha': [0.01, 0.1, 1, 10, 100, 1000]}


ridge_tuned = RandomizedSearchCV(ridge, param_grid,
                          cv = 3, 
                          scoring = ['neg_root_mean_squared_error', 'r2'], 
                          refit = 'neg_root_mean_squared_error',
                          verbose = 3,
                          n_iter = 25)

ridge_tuned.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1, neg_root_mean_squared_error=-516.762, r2=0.056, total=   1.8s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1, neg_root_mean_squared_error=-349.339, r2=0.112, total=   1.7s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s


[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=1, neg_root_mean_squared_error=-243.788, r2=0.183, total=   1.8s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1 
[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1, neg_root_mean_squared_error=-516.762, r2=0.056, total=   1.5s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1 
[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1, neg_root_mean_squared_error=-349.339, r2=0.112, total=   1.5s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1 
[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=1, Ridge__alpha=0.1, neg_root_mean_squared_error=-243.788, r2=0.183, total=   1.4s
[CV] encoder__smoothing=1.0, encoder__min_samples_leaf=3, Ridge__alpha=1 
[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=3, Ridge__alpha=1, neg_root_mean_squared_error=-518.283, r2=0.051, 

[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=2, Ridge__alpha=10, neg_root_mean_squared_error=-348.871, r2=0.114, total=   1.5s
[CV] encoder__smoothing=1.0, encoder__min_samples_leaf=2, Ridge__alpha=10 
[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=2, Ridge__alpha=10, neg_root_mean_squared_error=-244.119, r2=0.181, total=   1.4s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000, neg_root_mean_squared_error=-518.001, r2=0.052, total=   1.4s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000, neg_root_mean_squared_error=-349.417, r2=0.112, total=   1.8s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=1, Ridge__alpha=1000, neg_root_mean_squared_error=-243.516, r2=0.185, total=

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  2.2min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('encoder',
                                              TargetEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            min_samples_leaf=1,
                                                            return_df=True,
                                                            smoothing=1.0,
                                                            verbose=0)),
                                             ('Ridge',
                                              Ridge(alpha=1.0, copy_X=True,
                                                    fit_intercept=True,
                      

In [132]:
ridge_tuned.best_score_

-368.4911704987159

In [149]:
ridge_tuned.best_params_

{'encoder__smoothing': 0.1, 'encoder__min_samples_leaf': 3, 'Ridge__alpha': 10}

In [130]:
y_hat_ridge_tuned = ridge_tuned.predict(X_test)
print(f"""RMSE: {mean_squared_error(y_test, y_hat_ridge_tuned, squared=False):.4f}
R^2 score: {r2_score(y_test, y_hat_ridge_tuned):.4f}""")

RMSE: 171.0608
R^2 score: 0.3383


Zastosowanie regularyzacji L2 nie zmieniło wyniku.

### Lasso
Regresja z regularyzacją L1 (do funkcji straty dodawany jest składnik ograniczający wartość wag $\lambda \sum_{i=1}^n |w_i|$, gdzie $\sum_{i=1}^n |w_i|$ to suma wartości bezwzględnych wag i $\lambda $ jest parametrem). 

In [139]:
lasso = Pipeline(steps=[('encoder', TargetEncoder()),  
                     ('Lasso', Lasso())])  


param_grid = {'encoder__smoothing': [0.1, 1., 10., 100., 1000.],
              'encoder__min_samples_leaf': [1, 2, 3],
              'Lasso__alpha': [0.01, 0.1, 1, 10, 100, 1000]}


lasso_tuned = RandomizedSearchCV(lasso, param_grid,
                          cv = 3, 
                          scoring = ['neg_root_mean_squared_error', 'r2'], 
                          refit = 'neg_root_mean_squared_error',
                          n_iter = 25)

lasso_tuned.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('encoder',
                                              TargetEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            min_samples_leaf=1,
                                                            return_df=True,
                                                            smoothing=1.0,
                                                            verbose=0)),
                                             ('Lasso',
                                              Lasso(alpha=1.0, copy_X=True,
                                                    fit_intercept=True,
                      

In [141]:
lasso_tuned.best_score_

-368.4151410433715

In [148]:
lasso_tuned.best_params_

{'encoder__smoothing': 0.1,
 'encoder__min_samples_leaf': 3,
 'Lasso__alpha': 100}

In [140]:
y_hat_lasso_tuned = lasso_tuned.predict(X_test)
print(f"""RMSE: {mean_squared_error(y_test, y_hat_lasso_tuned, squared=False):.4f}
R^2 score: {r2_score(y_test, y_hat_lasso_tuned):.4f}""")

RMSE: 170.8879
R^2 score: 0.3397


Zastosowanie regularyzacji L1 nieznacznie poprawiło jakość modelu. 

### Inny model regresyjny - XGBoost

In [137]:
from xgboost import XGBRegressor

In [143]:
xgb = Pipeline(steps=[('encoder', TargetEncoder()),  
                     ('XGB', XGBRegressor())])  


param_grid = {'encoder__smoothing': [0.1, 1., 10., 100., 1000.],
              'encoder__min_samples_leaf': [1, 2, 3],
              'XGB__n_estimators': [25, 50, 100],
              'XGB__max_depth': [1, 2, 3]}


xgb_tuned = RandomizedSearchCV(xgb, param_grid,
                          cv = 3, 
                          scoring = ['neg_root_mean_squared_error', 'r2'], 
                          refit = 'neg_root_mean_squared_error', 
                          verbose=3, 
                          n_iter = 25)

xgb_tuned.fit(X_train, y_train)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1, neg_root_mean_squared_error=-516.061, r2=0.059, total=   3.8s
[CV] encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.8s remaining:    0.0s


[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1, neg_root_mean_squared_error=-347.084, r2=0.123, total=   3.5s
[CV] encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1 


...

[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=1, neg_root_mean_squared_error=-244.717, r2=0.177, total=   3.5s
[CV] encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2 
[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2, neg_root_mean_squared_error=-514.107, r2=0.066, total=   5.1s
[CV] encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2 
[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2, neg_root_mean_squared_error=-349.094, r2=0.113, total=   4.7s
[CV] encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2 
[CV]  encoder__smoothing=1.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2, neg_root_mean_squared_error=-240.986, r2=0.202, total=   6.4s
[CV] encoder__smoothing=1000.0, encoder__min_samples_le

[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2, neg_root_mean_squared_error=-347.311, r2=0.122, total=   5.5s
[CV] encoder__smoothing=1000.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2 
[CV]  encoder__smoothing=1000.0, encoder__min_samples_leaf=2, XGB__n_estimators=100, XGB__max_depth=2, neg_root_mean_squared_error=-235.037, r2=0.241, total=   5.8s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=3, XGB__n_estimators=100, XGB__max_depth=1 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=3, XGB__n_estimators=100, XGB__max_depth=1, neg_root_mean_squared_error=-516.077, r2=0.059, total=   5.3s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=3, XGB__n_estimators=100, XGB__max_depth=1 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=3, XGB__n_estimators=100, XGB__max_depth=1, neg_root_mean_squared_error=-347.695, r2=0.120, total=   4.7s
[CV] encoder__smoothing=10.0, encoder__min

[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=1, neg_root_mean_squared_error=-516.302, r2=0.058, total=   3.1s
[CV] encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=1 
[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=1, neg_root_mean_squared_error=-347.594, r2=0.121, total=   3.0s
[CV] encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=1 
[CV]  encoder__smoothing=0.1, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=1, neg_root_mean_squared_error=-244.289, r2=0.180, total=   2.8s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=3 
[CV]  encoder__smoothing=10.0, encoder__min_samples_leaf=2, XGB__n_estimators=50, XGB__max_depth=3, neg_root_mean_squared_error=-512.217, r2=0.073, total=   4.1s
[CV] encoder__smoothing=10.0, encoder__min_samples_leaf=2, X

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  5.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('encoder',
                                              TargetEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            min_samples_leaf=1,
                                                            return_df=True,
                                                            smoothing=1.0,
                                                            verbose=0)),
                                             ('XGB',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                       

In [146]:
xgb_tuned.best_params_

{'encoder__smoothing': 1000.0,
 'encoder__min_samples_leaf': 1,
 'XGB__n_estimators': 100,
 'XGB__max_depth': 3}

In [147]:
xgb_tuned.best_score_

-362.7094703594069

In [145]:
y_hat_xgb_tuned = xgb_tuned.predict(X_test)
print(f"""RMSE: {mean_squared_error(y_test, y_hat_xgb_tuned, squared=False):.4f}
R^2 score: {r2_score(y_test, y_hat_xgb_tuned):.4f}""")

RMSE: 177.0158
R^2 score: 0.2914


Na podstawie miar RMSE i $R^2$ najlepszym modelem okazała się regresja Lasso. XGBoost się przeuczył, osiągnął najlepszy wynik z kroswalidacji, ale najgorszy na zbiorze testowym.

Ciekawym jest też fakt, że optymalna wartość parametru smoothing okazywała się niewielka dla modeli liniowych, nawet mimo zastosowania kroswalidacji i target encodingu oddzielnie na zbiorze testowym (bez target leaku). Dopiero dla XGBoosta optymalna wartość była większa.  

Oczywiście należy mieć na uwadze, że różnice są niewielkie. Można też przemyśleć oddzielne ustawienia smoothingu dla każdej z kolumn. 